In [ ]:
import ast
import os
from pathlib import Path
from pprint import pprint
from typing import List, Tuple

import numpy as np
import padre_meddea
import pandas as pd
from astropy.io import fits
from astropy.time import Time
from padre_meddea import config
from padre_meddea.calibration.calibration import process_file
from padre_meddea.io.fits_tools import (  # New Stuff
    CUSTOM_ATTRS_PATH,
    _concatenate_input_files,
    _filter_hdul_time_ranges,
    _get_combined_list,
    _get_output_path,
    _init_hdul_structure,
    _sort_hdul_template,
    _write_output_file,
    get_comment,
    get_hdu_data_times,
    split_hdul_by_day,
    split_provenance_tables_by_day,
    update_hdul_date_metadata,
    update_hdul_filename_metadata,
)
from padre_meddea.util.util import (
    calc_time,
    create_science_filename,
    parse_science_filename,
)
from solarnet_metadata.schema import SOLARNETSchema
from solarnet_metadata.validation import validate_file, validate_header

In [2]:
def get_file_header_times(file_path: Path) -> Tuple[Time, Time]:
    """
    Get the DATE-BEG and DATE-END from the FITS file header.

    Parameters
    ----------
    file_path : Path
        Path to the FITS file.

    Returns
    -------
    Tuple[Time, Time]
        A tuple containing the start date (DATE-BEG) and end date (DATE-END) as astropy Time objects.

    Raises
    -------
    ValueError
        If the file does not contain DATE-BEG, DATE-END, or DATEREF keywords.
    """
    hdul = fits.open(file_path)
    header = hdul[0].header.copy()
    hdul.close()

    # Get Start Date
    if "DATE-BEG" in header:
        date_beg = Time(header["DATE-BEG"])
    elif "DATEREF" in header:
        date_beg = Time(header["DATEREF"])
    else:
        raise ValueError(f"File {file_path} does not contain DATE-BEG or DATEREF.")

    # Get End Date
    if "DATE-END" in header:
        date_end = Time(header["DATE-END"])
    elif "DATEREF" in header:
        date_end = Time(header["DATEREF"])
    else:
        raise ValueError(f"File {file_path} does not contain DATE-END or DATEREF.")

    return date_beg, date_end


def get_file_data_times(file_path: Path) -> Time:
    """
    Extract time information from the data within a FITS file.

    This function parses times differently based on the file descriptor (eventlist, hk, spec)
    extracted from the filename. It accesses the appropriate HDU and data columns for
    each file type to calculate accurate time values.

    Parameters
    ----------
    file_path : Path
        Path to the FITS file to extract time data from

    Returns
    -------
    Time
        Astropy Time object containing the time values extracted from the file data

    Raises
    ------
    ValueError
        If the file descriptor is not recognized or supported
    """
    # Get the File Desctiptor
    # We need to parse times differently for Photon / Spectrum / HK
    file_meta = parse_science_filename(file_path)
    file_descriptor = file_meta["descriptor"]
    times = None

    hdul = fits.open(file_path)
    # Calculate Times based on the file descriptor
    if file_descriptor == "photon":
        times = calc_time(
            hdul["SCI"].data["pkttimes"],
            hdul["SCI"].data["pktclock"],
            hdul["SCI"].data["clocks"],
        )
    elif file_descriptor == "housekeeping":
        times = calc_time(hdul["HK"].data["timestamp"])
    elif file_descriptor == "spectrum":
        times = calc_time(hdul["PKT"].data["pkttimes"], hdul["PKT"].data["pktclock"])
    else:
        raise ValueError(f"File contents of {file_path} not recogized.")
    # Explicitly Open and Close File - Windows Garbage Disposer cannot be trusted.
    hdul.close()

    return times

In [3]:
# Get Path to the Unit Test Data Directory
test_files_path = Path(padre_meddea.__file__).parent / "data" / "test"
print(test_files_path)
test_files_path.exists() or print("Test files path does not exist!")

# Make a testfiles path locally if it does not exist
cwd = Path.cwd()
results_path = cwd / "testfiles"
if not results_path.exists():
    results_path.mkdir(parents=True, exist_ok=True)

/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test


In [4]:
test_files = list(test_files_path.rglob("*.fits"))
file_meta = []
for filename in test_files:

    meta = parse_science_filename(filename.name)
    meta["path"] = filename.absolute()

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso

    file_meta.append(meta)

df = pd.DataFrame(file_meta)
df.sort_values(by=["descriptor", "time_beg"], inplace=True)
df

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
1,meddea,None,True,2025-05-04T05:51:38.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 05:51:38.000,2025-05-04 05:53:08.000
2,meddea,None,True,2025-05-04T05:53:08.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 05:53:38.000,2025-05-04 05:55:08.000
0,meddea,None,True,2025-05-04T05:55:08.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 05:55:38.000,2025-05-04 05:57:08.000
3,meddea,None,True,2025-05-04T05:57:08.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-05,2025-05-04 05:57:38.000,2025-05-05 05:59:08.000
11,meddea,None,True,2025-05-04T05:53:11.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 05:53:11.353,2025-05-04 05:53:11.830
8,meddea,None,True,2025-05-04T07:37:49.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 07:37:49.472,2025-05-04 07:37:49.476
10,meddea,None,True,2025-05-04T08:03:30.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-04,2025-05-04 08:03:30.385,2025-05-04 08:03:30.390
9,meddea,None,True,2025-05-04T08:32:34.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2025-05-04,2025-05-05,2025-05-04 08:32:34.299,2025-05-05 08:32:34.430
4,meddea,None,True,2025-05-04T10:38:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 10:39:51.392
5,meddea,None,True,2025-05-04T11:49:21.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/p...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:50:51.406


In [5]:
filename

PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/eventlist/padre_meddea_l0test_photon_20250504T055311_v0.1.0.fits')

# Second Table-Based Design

-  Possibility that Times will be wrong (i.e. times set to 0 or year 2000) We need to add some sort of check to throw out data from *not this year* Alternatively we can check that there is not too large of a jump in times. 
- It will probably be on a weekly basis that we'll re-downlink the corrupted files. There will be a person responsible for monitoring the validation outputs and bring these up in the weekly MOC-SOC operational meeting. 

Pseudocode:
1. Create a set of empty data structures or use default data structures from Existing file
2. For-Each input File
    - 2a Extract the data structures from the input file
    - 2b. Concatenate the file data structures into the rolling data structures
3. Calculate times for all rolling data structures
4. Sort the rolling data structures based on data times
     - Will we need to re-calculate the data times? Do we sort with a fancy lambda? Do we do some kind of argsort based on the data times?
5. Filter the rolling data structures based on time range checking
6. Split the rolling tables based on Day boundaries into separate DOY data structures
7. For-each DOY data structure
    - 7a. Re-calculate the metadata for each HDU
    - 7b. Convert the DOY data structures back into a HDUL
    - 7c. Save each HDUL to a unique output file. 

## Concat Photon Files

In [6]:
all_files = list(df[df["descriptor"] == "photon"]["path"])
all_files

[PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/eventlist/padre_meddea_l0test_photon_20250504T055311_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/eventlist/padre_meddea_l0test_photon_20250504T073749_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/eventlist/padre_meddea_l0test_photon_20250504T080330_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/eventlist/padre_meddea_l0test_photon_20250504T083234_v0.1.0.fits')]

In [7]:
# Change Current Directory to `testfiles`
os.chdir(cwd / "testfiles")

all_files = _get_combined_list(all_files, existing_file=None)

# Create new provenance table from the files to combine
provenance_tables = split_provenance_tables_by_day(all_files, existing_file=None)

# Initialize Data Structures
hdul_dict = _init_hdul_structure(all_files[0])

# Concatenate Input Files
hdul_dict = _concatenate_input_files(all_files[1:], hdul_dict)

# Sort Data Structures by Time
hdul_dict = _sort_hdul_template(hdul_dict)

# Filter HDUL baed on Time Range Checking
hdul_dict = _filter_hdul_time_ranges(
    hdul_dict=hdul_dict,
    start_time=Time("2025-01-01 00:00:00.000", format="iso", scale="utc"),
    end_time=Time("2050-01-01 00:00:00.000", format="iso", scale="utc"),
)

# Split HDU by Day
hdul_dicts = split_hdul_by_day(hdul_dict)

photon_outfiles = []
# Save each Day
for day, day_hdul in hdul_dicts.items():

    # Calculate the Outputn Path Filename
    outfile = _get_output_path(
        first_file=all_files[0], date_beg=Time(day + "T00:00:00")
    )

    # Update HDUL Primary Header with Date/Time Information
    day_hdul = update_hdul_date_metadata(day_hdul)

    # Update HDUL Primary Header with Filename Information
    day_hdul = update_hdul_filename_metadata(
        day_hdul, outfile, provenance_tables[day]
    )

    # Add Provenance Table to HDU
    if day in provenance_tables:
        prov_data = provenance_tables[day]

        prov_table = {
            "header": fits.Header(
                [
                    ("EXTNAME", "PROVENANCE", get_comment("EXTNAME")),
                    ("COMMENT", "Provenance information for the concatenated files"),
                    ("OBS_HDU", 0, get_comment("OBS_HDU")),
                ]
            ),
            "data": prov_data,
            "type": "bintable",
            "name": "PROVENANCE",
        }
        day_hdul[max(day_hdul) + 1] = prov_table

    # Write output file
    out_path = _write_output_file(day_hdul, outfile)

    photon_outfiles.append(Path(out_path))
    
os.chdir(cwd)

2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-BEG' types <class 'str'> and <class 'str'>, choosing DATE-BEG='2025-05-04T07:37:49.472'
2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-END' types <class 'str'> and <class 'str'>, choosing DATE-END='2025-05-04T07:37:49.476'
2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'ORIGFILE' types <class 'str'> and <class 'str'>, choosing ORIGFILE='PADREMDA0_250504073754.DAT'
2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-AVG' types <class 'str'> and <class 'str'>, choosing DATE-AVG='2025-05-04T07:37:49.474'
2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE' types <class 'str'> and <class 'str'>, choosing DATE='2025-06-17T17:14:20.773'
2025-06-23 14:27:12 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATEREF' types <class 's

INFO: Created concatenated daily file: padre_meddea_l1_photon_20250504T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


2025-06-23 14:27:13 - astropy - WARNING: VerifyWarning: Card is too long, comment will be truncated.
2025-06-23 14:27:13 - swxsoc - INFO: Created concatenated daily file: padre_meddea_l1_photon_20250505T000000_v0.1.0.fits


INFO: Created concatenated daily file: padre_meddea_l1_photon_20250505T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


In [8]:
photon_outfiles

[PosixPath('padre_meddea_l1_photon_20250504T000000_v0.1.0.fits'),
 PosixPath('padre_meddea_l1_photon_20250505T000000_v0.1.0.fits')]

## Concat Housekeeping Files

In [9]:
all_files = list(df[df["descriptor"] == "housekeeping"]["path"])
all_files

[PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/hk/padre_meddea_l0test_housekeeping_20250504T055138_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/hk/padre_meddea_l0test_housekeeping_20250504T055308_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/hk/padre_meddea_l0test_housekeeping_20250504T055508_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/hk/padre_meddea_l0test_housekeeping_20250504T055708_v0.1.0.fits')]

In [10]:
# Change Current Directory to `testfiles`
os.chdir(cwd / "testfiles")

all_files = _get_combined_list(all_files, existing_file=None)

# Create new provenance table from the files to combine
provenance_tables = split_provenance_tables_by_day(all_files, existing_file=None)

# Initialize Data Structures
hdul_dict = _init_hdul_structure(all_files[0])

# Concatenate Input Files
hdul_dict = _concatenate_input_files(all_files[1:], hdul_dict)

# Sort Data Structures by Time'
hdul_dict = _sort_hdul_template(hdul_dict)

# Filter HDUL baed on Time Range Checking
hdul_dict = _filter_hdul_time_ranges(
    hdul_dict=hdul_dict,
    start_time=Time("2025-01-01 00:00:00.000", format="iso", scale="utc"),
    end_time=Time("2050-01-01 00:00:00.000", format="iso", scale="utc"),
)

# Split HDU by Day
hdul_dicts = split_hdul_by_day(hdul_dict)

hk_outfiles = []
# Save each Day
for day, day_hdul in hdul_dicts.items():

    # Calculate the Outputn Path Filename
    outfile = _get_output_path(
        first_file=all_files[0], date_beg=Time(day + "T00:00:00")
    )

    # Update HDUL Primary Header with Date/Time Information
    day_hdul = update_hdul_date_metadata(day_hdul)

    # Update HDUL Primary Header with Filename Information
    day_hdul = update_hdul_filename_metadata(
        day_hdul, outfile, provenance_tables[day]
    )

    # Add Provenance Table to HDU
    if day in provenance_tables:
        prov_data = provenance_tables[day]

        prov_table = {
            "header": fits.Header(
                [
                    ("EXTNAME", "PROVENANCE", get_comment("EXTNAME")),
                    ("COMMENT", "Provenance information for the concatenated files"),
                    ("OBS_HDU", 0, get_comment("OBS_HDU")),
                ]
            ),
            "data": prov_data,
            "type": "bintable",
            "name": "PROVENANCE",
        }
        day_hdul[max(day_hdul) + 1] = prov_table

    # Write output file
    out_path = _write_output_file(day_hdul, outfile)

    hk_outfiles.append(Path(out_path))

os.chdir(cwd)

2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-BEG' types <class 'str'> and <class 'str'>, choosing DATE-BEG='2025-05-04T05:53:38.000'
2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATEREF' types <class 'str'> and <class 'str'>, choosing DATEREF='2025-05-04T05:53:38.000'
2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'CHECKSUM' types <class 'str'> and <class 'str'>, choosing CHECKSUM='XLINYIILXIILXIIL'
2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATASUM' types <class 'str'> and <class 'str'>, choosing DATASUM='292204281'
2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-END' types <class 'str'> and <class 'str'>, choosing DATE-END='2025-05-04T05:55:08.000'
2025-06-23 14:27:13 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'CHECKSUM' types <class 'str'> and <class 'st

INFO: Created concatenated daily file: padre_meddea_l1_housekeeping_20250504T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


2025-06-23 14:27:13 - astropy - WARNING: VerifyWarning: Card is too long, comment will be truncated.
2025-06-23 14:27:13 - swxsoc - INFO: Created concatenated daily file: padre_meddea_l1_housekeeping_20250505T000000_v0.1.0.fits


INFO: Created concatenated daily file: padre_meddea_l1_housekeeping_20250505T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


In [11]:
hk_outfiles

[PosixPath('padre_meddea_l1_housekeeping_20250504T000000_v0.1.0.fits'),
 PosixPath('padre_meddea_l1_housekeeping_20250505T000000_v0.1.0.fits')]

## Concat Spectrum Files

In [12]:
all_files = list(df[df["descriptor"] == "spectrum"]["path"])
all_files

[PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/spec/padre_meddea_l0test_spectrum_20250504T103811_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/spec/padre_meddea_l0test_spectrum_20250504T114921_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/spec/padre_meddea_l0test_spectrum_20250504T141211_v0.1.0.fits'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/PADRE_SOC/padre_meddea/padre_meddea/data/test/spec/padre_meddea_l0test_spectrum_20250504T070411_v0.1.0.fits')]

In [13]:
# Change Current Directory to `testfiles`
os.chdir(cwd / "testfiles")

all_files = _get_combined_list(all_files, existing_file=None)

# Create new provenance table from the files to combine
provenance_tables = split_provenance_tables_by_day(all_files, existing_file=None)

# Initialize Data Structures
hdul_dict = _init_hdul_structure(all_files[0])

# Concatenate Input Files
hdul_dict = _concatenate_input_files(all_files[1:], hdul_dict)

# Sort Data Structures by Time'
hdul_dict = _sort_hdul_template(hdul_dict)

# Filter HDUL baed on Time Range Checking
hdul_dict = _filter_hdul_time_ranges(
    hdul_dict=hdul_dict,
    start_time=Time("2025-01-01 00:00:00.000", format="iso", scale="utc"),
    end_time=Time("2050-01-01 00:00:00.000", format="iso", scale="utc"),
)

# Split HDU by Day
hdul_dicts = split_hdul_by_day(hdul_dict)

spec_outfiles = []
# Save each Day
for day, day_hdul in hdul_dicts.items():

    # Calculate the Outputn Path Filename
    outfile = _get_output_path(
        first_file=all_files[0], date_beg=Time(day + "T00:00:00")
    )

    # Update HDUL Primary Header with Date/Time Information
    day_hdul = update_hdul_date_metadata(day_hdul)

    # Update HDUL Primary Header with Filename Information
    day_hdul = update_hdul_filename_metadata(
        day_hdul, outfile, provenance_tables[day]
    )

    # Add Provenance Table to HDU
    if day in provenance_tables:
        prov_data = provenance_tables[day]

        prov_table = {
            "header": fits.Header(
                [
                    ("EXTNAME", "PROVENANCE", get_comment("EXTNAME")),
                    ("COMMENT", "Provenance information for the concatenated files"),
                    ("OBS_HDU", 0, get_comment("OBS_HDU")),
                ]
            ),
            "data": prov_data,
            "type": "bintable",
            "name": "PROVENANCE",
        }
        day_hdul[max(day_hdul) + 1] = prov_table

    # Write output file
    out_path = _write_output_file(day_hdul, outfile)

    spec_outfiles.append(Path(out_path))

os.chdir(cwd)

2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE' types <class 'str'> and <class 'str'>, choosing DATE='2025-06-17T17:15:01.735'
2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'FILENAME' types <class 'str'> and <class 'str'>, choosing FILENAME='padre_meddea_l0test_spectrum_20250504T114921_v0.1.0.fits'
2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'CHECKSUM' types <class 'str'> and <class 'str'>, choosing CHECKSUM='kZg1lZg0kZg0kZg0'
2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATASUM' types <class 'str'> and <class 'str'>, choosing DATASUM='2047846535'
2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-END' types <class 'str'> and <class 'str'>, choosing DATE-END='2025-05-04T11:50:51.406'
2025-06-23 14:27:23 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE' types <cla

INFO: Created concatenated daily file: padre_meddea_l1_spectrum_20250504T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


2025-06-23 14:27:23 - swxsoc - INFO: Created concatenated daily file: padre_meddea_l1_spectrum_20250505T000000_v0.1.0.fits


INFO: Created concatenated daily file: padre_meddea_l1_spectrum_20250505T000000_v0.1.0.fits [padre_meddea.io.fits_tools]


In [14]:
spec_outfiles

[PosixPath('padre_meddea_l1_spectrum_20250504T000000_v0.1.0.fits'),
 PosixPath('padre_meddea_l1_spectrum_20250505T000000_v0.1.0.fits')]

In [15]:
for file in spec_outfiles:
    # Display HDUL Info for each file
    hdul = fits.open("testfiles" / file)
    pprint(hdul.info())
    hdul.close()

Filename: testfiles/padre_meddea_l1_spectrum_20250504T000000_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      38   ()      
  1  SPEC          1 ImageHDU        31   (512, 24, 31)   float64   
  2  PKT           1 BinTableHDU     49   31R x 5C   [J, J, 24I, 24I, I]   
  3  PROVENANCE    1 BinTableHDU     19   4R x 3C   [56A, 23A, 23A]   
None
Filename: testfiles/padre_meddea_l1_spectrum_20250505T000000_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      38   ()      
  1  SPEC          1 ImageHDU        31   (512, 24, 5)   float64   
  2  PKT           1 BinTableHDU     49   5R x 5C   [J, J, 24I, 24I, I]   
  3  PROVENANCE    1 BinTableHDU     19   1R x 3C   [56A, 23A, 23A]   
None


## Check for SOLARNET Compliance in the L1 Data

In [16]:
outfiles = photon_outfiles + hk_outfiles + spec_outfiles

In [17]:
# Create Custome PADRE SOLARNET schema
padre_schema = SOLARNETSchema(schema_layers=[CUSTOM_ATTRS_PATH])

# Change Current Directory to `testfiles`
os.chdir(cwd / "testfiles")

files = []
all_findings = []
for processed_file in outfiles:
    # Validate the first Processed File against the SOALRNET schema
    file_findings = validate_file(
        file_path=processed_file,
        warn_empty_keyword=True,
        warn_no_comment=False,
        warn_data_type=True,
        schema=padre_schema,
    )
    all_findings.extend(file_findings)
    files.extend([processed_file.name] * len(file_findings))
    
os.chdir(cwd)

2025-06-23 14:27:24 - solarnet_metadata.validation - WARNING: Keyword `OBS_HDU` is set to 0, but `is_obs` given as True. Overriding `is_obs` to False. If this is not the desired behavior, please check the header `OBS_HDU`.
2025-06-23 14:27:24 - solarnet_metadata.validation - WARNING: Keyword `OBS_HDU` is set to 0, but `is_obs` given as True. Overriding `is_obs` to False. If this is not the desired behavior, please check the header `OBS_HDU`.
2025-06-23 14:27:24 - solarnet_metadata.validation - WARNING: Keyword `OBS_HDU` is set to 0, but `is_obs` given as True. Overriding `is_obs` to False. If this is not the desired behavior, please check the header `OBS_HDU`.
2025-06-23 14:27:24 - solarnet_metadata.validation - WARNING: Keyword `OBS_HDU` is set to 0, but `is_obs` given as True. Overriding `is_obs` to False. If this is not the desired behavior, please check the header `OBS_HDU`.
2025-06-23 14:27:24 - solarnet_metadata.validation - WARNING: Keyword `OBS_HDU` is set to 0, but `is_obs` gi

In [18]:
df = pd.DataFrame([files, all_findings]).T
df.columns = ["file", "findings"]

# Group by findings and get unique filenames for each finding
findings_summary = df.groupby('findings')['file'].unique().reset_index()

# Optionally, add a count of files for each finding
findings_summary['file_count'] = findings_summary['file'].apply(len)

# Sort by most common findings first
findings_summary = findings_summary.sort_values('file_count', ascending=True).reset_index(drop=True)

findings_summary

,findings,file,file_count
0,Observation Header 2: Value for 'DATE-BEG' can...,[padre_meddea_l1_housekeeping_20250505T000000_...,1
1,Primary Header: FITS card for 'PARENTXT' excee...,[padre_meddea_l1_spectrum_20250504T000000_v0.1...,1
2,Primary Header: FITS card for 'PARENTXT' excee...,[padre_meddea_l1_photon_20250504T000000_v0.1.0...,1
3,Primary Header: FITS card for 'PARENTXT' excee...,[padre_meddea_l1_housekeeping_20250504T000000_...,1
4,Observation Header 2: Value for 'DATEREF' cann...,[padre_meddea_l1_housekeeping_20250505T000000_...,1
5,Observation Header 2: Value for 'DATE-END' can...,[padre_meddea_l1_housekeeping_20250505T000000_...,1
6,Observation Header 2: Value for 'DATE-AVG' can...,[padre_meddea_l1_housekeeping_20250505T000000_...,1
7,Observation Header 2: Keyword 'TUNIT1' not fou...,[padre_meddea_l1_housekeeping_20250504T000000_...,2
8,Observation Header 2: Keyword 'TREFPOS' not fo...,[padre_meddea_l1_housekeeping_20250504T000000_...,2
9,Observation Header 2: Keyword 'JDREF' not foun...,[padre_meddea_l1_housekeeping_20250504T000000_...,2


In [19]:
pprint(findings_summary["findings"].values)

array(["Observation Header 2: Value for 'DATE-BEG' cannot be cast to data type 'date': Invalid isoformat string: ''",
       "Primary Header: FITS card for 'PARENTXT' exceeds 80 characters (length: 277).",
       "Primary Header: FITS card for 'PARENTXT' exceeds 80 characters (length: 269).",
       "Primary Header: FITS card for 'PARENTXT' exceeds 80 characters (length: 293).",
       "Observation Header 2: Value for 'DATEREF' cannot be cast to data type 'date': Invalid isoformat string: ''",
       "Observation Header 2: Value for 'DATE-END' cannot be cast to data type 'date': Invalid isoformat string: ''",
       "Observation Header 2: Value for 'DATE-AVG' cannot be cast to data type 'date': Invalid isoformat string: ''",
       "Observation Header 2: Keyword 'TUNIT1' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'TREFPOS' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'JDREF' not found in the